In [ ]:
import requests as req
from bs4 import BeautifulSoup as bs
from datetime import datetime
import csv

hades = { 'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36'}

In [ ]:
def scrape_cnbc(hal):
  global hades
  a = 1
  for page in range(1, hal+1):
    url_cnbc = f"https://www.cnbcindonesia.com/search?query=bea%20cukai&p={page}&kanal=&tipe=artikel&date="
    ge = req.get(url_cnbc, headers=hades).text
    sop = bs(ge, 'lxml')
    ul = sop.find('ul', class_='list media_rows middle thumb terbaru gtm_indeks_feed')
    if ul is None:
      continue
    li = ul.find_all('li')
    for x in li:
      link_art = x.find('article').find('a')['href']
      ge_ = req.get(link_art, headers=hades).text
      sop_ = bs(ge_, 'lxml')
      art_div = sop_.find('div', class_='lm_content mt10')
      if art_div is None:
        continue
      art = art_div.find('article')
      if art is None:
        continue
      headline = art.find('h1').text
      date_text = art.find('div', class_='date').text
      date = datetime.strptime(date_text, '%d %B %Y %H:%M')
      warp = art.find_all('div', class_='detail_text')
      for w in warp:
        paragraphs = w.find_all('p')
        content = ''.join(p.text for p in paragraphs).replace('\n', '').replace('ADVERTISEMENT', '').replace('SCROLL TO RESUME CONTENT', '')
        print(f'done[{a}] > {headline[0:10]}')
        a += 1
        with open('cnbc_result.csv', 'a', newline='', encoding='utf-8') as file:
          wr = csv.writer(file, delimiter=',')
          wr.writerow([headline, date, link_art, content])

In [ ]:
scrape_cnbc(4)